In [ ]:
import json
import pymysql
from itertools import combinations

region_name = 'us-east-1b'
conn = None
        
def openConnection():
    '''Call to open connection. 
       For privacy, username, password, and DB names have been anonymized.
       If you would like to replicate this code, 
       you can fill those in with your own values'''
    
    global conn
    name = 'USERNAME'
    password = 'PASSWORD'
    db_name = 'DB_NAME'
    rds_host = 'DB_HOST_NAME'

    if(conn is None):
        conn = pymysql.connect(
            rds_host, user=name, passwd=password, db=db_name, connect_timeout=3)
    elif (not conn.open):
        conn = pymysql.connect(
            rds_host, user=name, passwd=password, db=db_name, connect_timeout=3)


def lambda_handler(event, context):
    '''Query RDS several times, counting the number of songs with these tags in RDS.
        Formats for input into the Venn Diagram chart.
    
       Inputs: event from API Gateway containing queryStringParameters.
               queryStringParameters = python dict with arbitrary number of tags'''
    
    #retrieve tags from event string
    request_l = event['queryStringParameters'].values()
    
    #for testing purposes, can uncomment this to test these hardcoded tags
    # request_l = ['pop','90s','rock']

    #open that connection 
    openConnection()
    with conn.cursor() as cur:
        #this section will query RDS many times, looping through the combinations of tags
        for i in range(1, len(request_l) + 1):
            #if only one tag, the this will run once
            #if two tags, runs three times,
            #if three tags, runs 6 times, each with new combination
            for x in combinations(request_l, i):
                #this line will yield something like:
                ##"TAGS LIKE '%foo%' AND TAGS LIKE '%bar%' AND TAGS LIKE '%baz%'"
                q = ' AND '.join(["TAGS LIKE '%{}%'".format(s) for s in x])
                #perform query, pasting in your query from this loop
                cur.execute("select COUNT(SONG_ID) from TRACKS_WITH_TAGS WHERE {}".format(q))
                #and retrieve size of query
                for row in cur:
                    size = row[0]
                #format for return to front-end
                labels = " & ".join([s.title() for s in x])
                sets = ["#"+s for s in x]
                #and append
                ret_json.append({'sets':sets, 'label':labels, 'size': size})
    conn.close()
    return {
        'statusCode': 200,
        'body': json.dumps(ret_json)
    }
